In [58]:
import pandas as pd

In [59]:
df = pd.read_csv('../data/prospectuses_data.csv')

# Get RmsId without suffixes
df['RmsId'] = df['Prospectus ID'].str.split('_').str[0]
df['RmsId'] = df['RmsId'].astype(int)
df.head()

,Prospectus ID,Original Filename,Section ID,Section Title,Subsection ID,Subsection Title,Subsubsection ID,Subsubsection Title,Subsubsection Text,Parsing Error,From Folder,Prospectus Year,RmsId
0,5,Preliminary Offerings 2021.pdf,failed parsing,failed parsing,failed parsing,failed parsing,failed parsing,failed parsing,failed parsing,Exception occurred: local variable 'section_id...,raw_manual,2021,5
1,5_1,Offerings 2020.pdf,1,RISK FACTORS,1.1,Risks related to the ADLER Group’s Business Ac...,1.1.1,Our business is significantly dependent on our...,We rely significantly on rental income. In the...,NaN,raw_manual,2020,5
2,5_1,Offerings 2020.pdf,1,RISK FACTORS,1.1,Risks related to the ADLER Group’s Business Ac...,1.1.1,Our business is significantly dependent on our...,We rely significantly on rental income. In the...,NaN,raw_manual,2020,5
3,5_1,Offerings 2020.pdf,1,RISK FACTORS,1.1,Risks related to the ADLER Group’s Business Ac...,1.1.2,Our ability to operate our business successful...,We face the risk that we may not be able to ge...,NaN,raw_manual,2020,5
4,5_1,Offerings 2020.pdf,1,RISK FACTORS,1.1,Risks related to the ADLER Group’s Business Ac...,1.1.3,We rely on our ability to identify potential r...,"As part of our strategy, we evaluate real esta...",NaN,raw_manual,2020,5


### Merge with issuer data from DB

In [60]:
from capfourpy.databases import Database
db = Database(database="C4DW")

issuer_data_query = """
WITH RankedIssuerData AS (
  SELECT 
    RmsId, 
    AbbrevName, 
    CapFourIndustry, 
    CountryIsoOperating,
    ROW_NUMBER() OVER (PARTITION BY RmsId ORDER BY AbbrevName ASC) AS rn
  FROM [C4DW].[DailyOverview].[IssuerData]
  WHERE RmsId IS NOT NULL 
    AND RmsId > 0
	AND CapFourIndustry IS NOT NULL
	AND CountryIsoOperating IS NOT NULL
)
SELECT 
  RmsId, 
  AbbrevName, 
  CapFourIndustry, 
  CountryIsoOperating
FROM RankedIssuerData
WHERE rn = 1;
"""

data_database = db.read_sql(issuer_data_query)

# Merge data from df with data from data_database left on RmsId
df = df.merge(data_database, on='RmsId', how='left')
df


,Prospectus ID,Original Filename,Section ID,Section Title,Subsection ID,Subsection Title,Subsubsection ID,Subsubsection Title,Subsubsection Text,Parsing Error,From Folder,Prospectus Year,RmsId,AbbrevName,CapFourIndustry,CountryIsoOperating
0,5,Preliminary Offerings 2021.pdf,failed parsing,failed parsing,failed parsing,failed parsing,failed parsing,failed parsing,failed parsing,Exception occurred: local variable 'section_id...,raw_manual,2021,5,Adler Group,Real Estate,DE
1,5_1,Offerings 2020.pdf,1,RISK FACTORS,1.1,Risks related to the ADLER Group’s Business Ac...,1.1.1,Our business is significantly dependent on our...,We rely significantly on rental income. In the...,NaN,raw_manual,2020,5,Adler Group,Real Estate,DE
2,5_1,Offerings 2020.pdf,1,RISK FACTORS,1.1,Risks related to the ADLER Group’s Business Ac...,1.1.1,Our business is significantly dependent on our...,We rely significantly on rental income. In the...,NaN,raw_manual,2020,5,Adler Group,Real Estate,DE
3,5_1,Offerings 2020.pdf,1,RISK FACTORS,1.1,Risks related to the ADLER Group’s Business Ac...,1.1.2,Our ability to operate our business successful...,We face the risk that we may not be able to ge...,NaN,raw_manual,2020,5,Adler Group,Real Estate,DE
4,5_1,Offerings 2020.pdf,1,RISK FACTORS,1.1,Risks related to the ADLER Group’s Business Ac...,1.1.3,We rely on our ability to identify potential r...,"As part of our strategy, we evaluate real esta...",NaN,raw_manual,2020,5,Adler Group,Real Estate,DE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14725,1994,Preliminary Offerings 2024.pdf,1,RISK FACTORS,1.6,Risks Related to Our Indebtedness and the Notes,1.6.35,Unless the Acquisition will be consummated on ...,Unless the Acquisition will be consummated on ...,NaN,raw_manual,2024,1994,Truist Insurance,Insurance,US
14726,1994,Preliminary Offerings 2024.pdf,1,RISK FACTORS,1.6,Risks Related to Our Indebtedness and the Notes,1.6.36,If this offering closes prior to the consummat...,If this offering closes prior to the consummat...,NaN,raw_manual,2024,1994,Truist Insurance,Insurance,US
14727,1994,Preliminary Offerings 2024.pdf,1,RISK FACTORS,1.6,Risks Related to Our Indebtedness and the Notes,1.6.37,"In a bankruptcy proceeding, the holders of not...",If either of the Escrow Issuers commences a ba...,NaN,raw_manual,2024,1994,Truist Insurance,Insurance,US
14728,1994,Preliminary Offerings 2024.pdf,1,RISK FACTORS,1.7,NaN,1.7.1,NaN,_Investing in our notes involves a high degree...,NaN,raw_manual,2024,1994,Truist Insurance,Insurance,US


### Basic Statistics

In [61]:
# Print Unique RmsIds
print(f"Unique RmsIds: {df['RmsId'].nunique()}")

Unique RmsIds: 199


In [62]:
# Identify the largest suffix per RmsId
df['Suffix'] = df['Prospectus ID'].str.split('_').str[1].fillna('0').astype(int)
largest_suffix_df = df.loc[df.groupby('RmsId')['Suffix'].idxmax()]

# Filter rows with "failed parsing" in 'Section ID'
failed_parsing_df = largest_suffix_df[largest_suffix_df['Section ID'] == 'failed parsing']
print(f"Count of distinct RmsId with 'failed parsing' in the largest suffix or single ID: ")
print(failed_parsing_df['RmsId'].nunique())

# Filter rows where "Section ID" is NOT "failed parsing"
non_failed_parsing_df = largest_suffix_df[largest_suffix_df['Section ID'] != 'failed parsing']
print(f"Count of distinct RmsId without 'failed parsing' in the largest suffix or single ID: ")
print(non_failed_parsing_df['RmsId'].nunique())



Count of distinct RmsId with 'failed parsing' in the largest suffix or single ID: 
21
Count of distinct RmsId without 'failed parsing' in the largest suffix or single ID: 
178


**Countries**

In [73]:
# Count distinct RmsId within each CountryIsoOperating
distinct_rmsid_country = df.groupby('CountryIsoOperating')['RmsId'].nunique().reset_index()
distinct_rmsid_country.rename(columns={'RmsId': 'Distinct_RmsId_Count'}, inplace=True)

distinct_rmsid_country

,CountryIsoOperating,Distinct_RmsId_Count
0,AE,1
1,BE,1
2,BS,1
3,CA,5
4,CH,3
5,DE,17
6,DK,4
7,ES,4
8,FI,2
9,FR,13


**Industries**

In [66]:
# Count distinct RmsId within each CapFourIndustry
distinct_rmsid_industry = df.groupby('CapFourIndustry')['RmsId'].nunique().reset_index()
distinct_rmsid_industry.rename(columns={'RmsId': 'Distinct_RmsId_Count'}, inplace=True)

print(distinct_rmsid_industry)

             CapFourIndustry  Distinct_RmsId_Count
0                 Automotive                     9
1             Basic Industry                    26
2              Capital Goods                    19
3             Consumer Goods                    14
4         Financial Services                     7
5                 Healthcare                    18
6                  Insurance                     6
7                    Leisure                    13
8                      Media                    10
9                Real Estate                     4
10                    Retail                    17
11                  Services                    24
12  Technology & Electronics                    11
13        Telecommunications                    16
14            Transportation                     2
15                   Utility                     3


**Years**

In [65]:
# Count distinct RmsId within each Prospectus Year
distinct_rmsid_year = df.groupby('Prospectus Year')['RmsId'].nunique().reset_index()
distinct_rmsid_year.rename(columns={'RmsId': 'Distinct_RmsId_Count'}, inplace=True)

print(distinct_rmsid_year)

    Prospectus Year  Distinct_RmsId_Count
0              2007                     1
1              2010                     1
2              2012                     4
3              2013                     3
4              2014                     7
5              2015                     7
6              2016                    11
7              2017                    13
8              2018                    17
9              2019                    15
10             2020                    42
11             2021                    77
12             2022                    15
13             2023                    30
14             2024                    39


### Document Statistics

In [71]:
# Add a column for the length of 'Subsubsection Title'
df['Subsubsection_Title_Length'] = df['Subsubsection Title'].astype(str).str.len()
df['Subsubsection_Text_Length'] = df['Subsubsection Text'].astype(str).str.len()

df[['Subsubsection Title', 'Subsubsection_Title_Length', 'Subsubsection Text', 'Subsubsection_Text_Length']].head()

,Subsubsection Title,Subsubsection_Title_Length,Subsubsection Text,Subsubsection_Text_Length
0,failed parsing,14,failed parsing,14
1,Our business is significantly dependent on our...,209,We rely significantly on rental income. In the...,365
2,Our business is significantly dependent on our...,209,We rely significantly on rental income. In the...,2591
3,Our ability to operate our business successful...,413,We face the risk that we may not be able to ge...,4817
4,We rely on our ability to identify potential r...,345,"As part of our strategy, we evaluate real esta...",6745


In [72]:
# Group by 'Prospectus ID' and count the number of rows
prospectus_counts = df.groupby('Prospectus ID').size().reset_index(name='Row_Count')

# Calculate minimum, average, and maximum
min_count = prospectus_counts['Row_Count'].min()
avg_count = prospectus_counts['Row_Count'].mean()
max_count = prospectus_counts['Row_Count'].max()

print(f"Minimum number of rows per Prospectus ID: {min_count}")
print(f"Average number of rows per Prospectus ID: {avg_count:.2f}")
print(f"Maximum number of rows per Prospectus ID: {max_count}")

Minimum number of rows per Prospectus ID: 1
Average number of rows per Prospectus ID: 40.92
Maximum number of rows per Prospectus ID: 207
